In [1]:
import pandas as pd
import tqdm
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
phen=["c02x","r05x"]

In [3]:
pascode_col="PAC score - AD vs. Control"
# PASCODE columns:['PAC score - AD vs. Control',
       # 'PAC score - Sleep_WeightGain_Guilt_Suicide vs. Control',
       # 'PAC score - WeightLoss_PMA', 'PAC score - Depression Mood vs. Control',
       # 'PAC score - Braak stages']

df = pd.read_csv("/home/ubuntu/PASCODE/PAC_score.csv", index_col=0)
df_subset = df.dropna(subset = [f"{pascode_col}"])

donors = df_subset.index.str.split("-", n=1).str[0]
donors = list(set(donors))
df_subset.loc[:,"donor"] = df_subset.index.str.split("-", n=1).str[0]

clinical_meta = pd.read_csv("/home/ubuntu/metadata_latest_oct5.csv",low_memory=False)

/tmp/ipykernel_33770/299907152.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.loc[:,"donor"] = df_subset.index.str.split("-", n=1).str[0]


In [89]:
pac_donors = pd.DataFrame()
for donor in tqdm.tqdm(donors):
    pascode_subset = df_subset[df_subset["donor"]==donor]
    donor_phenotype = clinical_meta[clinical_meta["SubID"]==donor][f"{phen[0]}"].values[0]

    if donor_phenotype=="AD":
        pac_donors = pd.concat([pac_donors, pascode_subset.loc[pascode_subset[f"{pascode_col}"]>=0.5]])
    elif donor_phenotype=="Control":
        pac_donors = pd.concat([pac_donors, pascode_subset.loc[pascode_subset[f"{pascode_col}"]<=-0.5]])

if isinstance(phen,str):
    pac_donors_meta = pd.merge(pac_donors, clinical_meta[[f"{phen}","SubID"]], left_on="donor", right_on="SubID")
elif isinstance(phen, list):
    pac_donors_meta = pd.merge(pac_donors, clinical_meta[phen + ["SubID"]], left_on="donor", right_on="SubID")
pac_donors_meta.index = pac_donors.index

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 425/425 [00:55<00:00,  7.70it/s]


In [90]:
subset_phen="r05x"

In [91]:
pac_donors_meta = pac_donors_meta.dropna(subset=[subset_phen])
pac_donors_meta

,PAC score - AD vs. Control,PAC score - Sleep_WeightGain_Guilt_Suicide vs. Control,PAC score - WeightLoss_PMA,PAC score - Depression Mood vs. Control,PAC score - Braak stages,donor,c02x,r05x,SubID
M4030-1-AAGACTCCATCGATAC-0,0.744563,-0.005258,0.229006,-0.010087,0.964105,M4030,AD,1.0,M4030
M4030-1-AATCGTGGTGATACCT-0,0.531768,0.320565,0.845926,0.021009,0.955168,M4030,AD,1.0,M4030
M4030-1-AATGGCTCAACAGAGC-0,0.556864,-0.536576,-0.710111,-0.025550,0.999078,M4030,AD,1.0,M4030
M4030-1-ACAACCAAGGCTATCT-0,0.868315,-0.054615,-0.114644,-0.294118,0.984806,M4030,AD,1.0,M4030
M4030-1-ACCGTTCTCACGGGCT-0,0.546648,-0.362885,-0.754779,0.076548,0.979500,M4030,AD,1.0,M4030
...,...,...,...,...,...,...,...,...,...
M56039-2-TGGGATTCAGCCATTA-1,0.796623,-0.687523,-0.855276,-0.726155,0.019436,M56039,AD,3.0,M56039
M56039-2-TGTGTGAGTTGTCCCT-1,0.678059,-0.078114,-0.411210,-0.004230,-0.135776,M56039,AD,3.0,M56039
M56039-2-TTACGCCGTGGCTGAA-1,0.556628,-0.232408,-0.472485,-0.781718,0.894771,M56039,AD,3.0,M56039
M56039-2-TTCGGTCAGCCTAGGA-1,0.573020,-0.078745,-0.656016,-0.001713,-0.130125,M56039,AD,3.0,M56039


In [92]:
# 1. donor-level summary
all_donors = (
    pac_donors_meta.groupby('donor')
      .agg(cell_count=('donor','size'),
           phenotype=(f"{subset_phen}",'first'))
      .reset_index()
)

# 2. bin into quantiles (here 4 bins; adjust as you like)
all_donors['count_bin'] = pd.qcut(all_donors['cell_count'],
                              q=4,
                              labels=False,
                              duplicates='drop')

# 3. stratify on “phenotype_bin”
all_donors['stratify_grp'] = all_donors['phenotype'].astype(str) + "_" + all_donors['count_bin'].astype(str)

train_donors, val_donors = train_test_split(
    all_donors,
    test_size=0.2,
    stratify=all_donors['stratify_grp'],
    random_state=42
)

# 4. build your splits
train_df = pac_donors_meta[pac_donors_meta['donor'].isin(train_donors['donor'])]
val_df   = pac_donors_meta[pac_donors_meta['donor'].isin(val_donors  ['donor'])]

In [93]:
print("Cells (train):", len(train_df), "  Cells (val):", len(val_df))
print("Phenotype ratio (train):")
print(train_df[f"{subset_phen}"].value_counts(normalize=True),train_df[f"{subset_phen}"].value_counts())
print("Phenotype ratio (val):")
print(val_df[f"{subset_phen}"].value_counts(normalize=True), val_df[f"{subset_phen}"].value_counts())

train_df = train_df[["SubID",f"{subset_phen}"]]
train_df["barcodekey"] = train_df.index
val_df = val_df[["SubID",f"{subset_phen}"]]
val_df["barcodekey"] = val_df.index

Cells (train): 278665   Cells (val): 64403
Phenotype ratio (train):
3.0    0.291791
5.0    0.232803
0.0    0.194004
4.0    0.122000
2.0    0.105636
1.0    0.053767
Name: r05x, dtype: float64 3.0    81312
5.0    64874
0.0    54062
4.0    33997
2.0    29437
1.0    14983
Name: r05x, dtype: int64
Phenotype ratio (val):
5.0    0.277922
3.0    0.227940
0.0    0.189851
2.0    0.122494
4.0    0.109684
1.0    0.072108
Name: r05x, dtype: float64 5.0    17899
3.0    14680
0.0    12227
2.0     7889
4.0     7064
1.0     4644
Name: r05x, dtype: int64


In [84]:
map_phen = {
    0: "Early",
    1: "Early",
    2: "Early",
    3: "Mid",
    4: "Mid",
    5: "Late",
    6: "Late",
}  

train_df[subset_phen] = train_df[subset_phen].map(map_phen)
val_df[subset_phen] = val_df[subset_phen].map(map_phen)

map_phen = {
    "Early": 0,
    "Mid":1,
    "Late":2,
}  

train_df[subset_phen] = train_df[subset_phen].map(map_phen)
val_df[subset_phen] = val_df[subset_phen].map(map_phen)

In [85]:
data={"train": train_df, "test": val_df}

In [86]:
train_df

,SubID,r01x,barcodekey
M66936-1-AAACCCATCTAACGGT-0,M66936,0,M66936-1-AAACCCATCTAACGGT-0
M66936-1-AAAGTCCTCAACCCGG-0,M66936,0,M66936-1-AAAGTCCTCAACCCGG-0
M66936-1-AACAAAGAGGGAGAAT-0,M66936,0,M66936-1-AACAAAGAGGGAGAAT-0
M66936-1-AACACACTCGCTGACG-0,M66936,0,M66936-1-AACACACTCGCTGACG-0
M66936-1-AAGCATCCATACAGGG-0,M66936,0,M66936-1-AAGCATCCATACAGGG-0
...,...,...,...
M56039-2-TGGGATTCAGCCATTA-1,M56039,2,M56039-2-TGGGATTCAGCCATTA-1
M56039-2-TGTGTGAGTTGTCCCT-1,M56039,2,M56039-2-TGTGTGAGTTGTCCCT-1
M56039-2-TTACGCCGTGGCTGAA-1,M56039,2,M56039-2-TTACGCCGTGGCTGAA-1
M56039-2-TTCGGTCAGCCTAGGA-1,M56039,2,M56039-2-TTCGGTCAGCCTAGGA-1


In [87]:
with open(f"/home/ubuntu/scripts/Train_cv_to_send/Phenotype_classification_files/{subset_phen}_split_seed42_PAC.pkl", 'wb') as handle:
    pickle.dump(data,handle)